In [1]:
# --- Imports ---
import os
import numpy as np
import skimage.io as io
import skimage.filters as filters
import skimage.morphology as morphology
import skimage.measure as measure
import skimage.segmentation as segmentation
import skimage.feature as feature
import scipy.ndimage as ndi
import bioio
import bioio_tifffile
import napari
from skimage.measure import regionprops
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=UserWarning)


# Load images


In [44]:
# --- REPLACE THIS SECTION WITH YOUR ACTUAL IMAGE LOADING ---
# Example: Load a 3D image from a file (e.g., TIFF stack)
# image = io.imread('path/to/your/3d_image.tif')
# If your image is (Y, X, Z) or (X, Y, Z), you might need to transpose it:
# image = np.transpose(image, (2, 0, 1)) # Example for (Y, X, Z) to (Z, Y, X)
#filename = '/home/mira/Documents/MASTER/MASTER THESIS/napari/images/crop_2306/20240821_1273_E_late_15um_02_n2v_cropped.tif'
filename = '/home/mira/Documents/MASTER/MASTER THESIS/napari/images/20241107_1268_E_30minHS_3h_5min_5um_n2v_t26.tif'
image_path = os.path.basename(filename)

img_3D = bioio.BioImage(filename, reader=bioio_tifffile.Reader)
image= img_3D.get_image_data("ZYX", C=1).astype(np.float32)
    
print(f"Loaded image shape: {image.shape}")
print(f"Image data type: {image.dtype}")
print(f"Image min/max intensity: {image.min()}/{image.max()}")

Loaded image shape: (26, 1000, 1000)
Image data type: float32
Image min/max intensity: 99.071044921875/275.77728271484375


# Pre processing

In [45]:
# --- Step 3A: Binarization / Initial Segmentation ---
# You'll likely need to adjust the thresholding method and parameters.

# 1. Global Thresholding (simple, but sensitive to intensity variations)
# Adjust `threshold_value` based on your image intensity histogram.
threshold_value = filters.threshold_otsu(image) # Otsu's method is a good starting point
# threshold_value = 500 # You might manually set a value
binary_image_global = image > threshold_value
print(f"Otsu threshold: {threshold_value}")

# 2. Local/Adaptive Thresholding (better for uneven illumination)
# `block_size`: Size of neighborhood to calculate local threshold. Must be odd.
# `offset`: A constant subtracted from the mean or weighted mean.
# Be careful with `block_size` in 3D; larger blocks are slower.
# For 3D, you might consider slice-by-slice adaptive thresholding if performance is an issue.
block_size_3D = 31 # Example: Must be odd. Adjust based on your object size.
offset_value = 0.05 * image.max() # Example offset, adjust as needed

# Apply adaptive thresholding. `mean` and `gaussian` are common methods.
# For 3D, `adaptive` method from `skimage.filters` can be slow.
# If it's too slow, consider applying `threshold_local` slice by slice, then stacking.
# try:
#     # This can be very slow for large 3D images; consider processing 2D slices if needed.
#     binary_image_adaptive = image > filters.threshold_local(
#         image, block_size=block_size_3D, method='gaussian', offset=offset_value
#     )
# except RuntimeError as e:
#     print(f"Warning: 3D adaptive thresholding might be too slow or memory intensive: {e}")
#     print("Consider processing slice-by-slice if this is the case.")
#     binary_image_adaptive = np.copy(binary_image_global) # Fallback

# Choose which binary image to proceed with for morphology
binary_image = binary_image_global # Start with global for simplicity, then try adaptive


Otsu threshold: 116.67263793945312


In [46]:
# --- Step 3B: Morphological Operations ---
# Experiment with different structuring element shapes and sizes.
# Structuring elements can be `disk` (2D), `ball` (3D sphere), or custom arrays.

# Define a 3D structuring element (e.g., a ball/sphere)
# `radius`: Controls the size of the structuring element. Crucial parameter!
selem_radius = 4 # Adjust this radius!
selem = morphology.ball(selem_radius)

# 1. Opening: Erosion followed by Dilation.
#   Purpose: Removes small objects and breaks thin connections between larger objects.
#   Useful for separating your weakly connected "spots".
binary_image_opened = morphology.binary_opening(binary_image, footprint=selem)


# 2. Closing: Dilation followed by Erosion.
#   Purpose: Fills small holes within objects and connects nearby objects separated by small gaps.
#   Useful if your "connected spots" have tiny breaks you want to bridge.
#   Apply after opening if you want to first separate, then fill within the separated parts.
# selem_close_radius = 1 # Often a smaller radius than opening
# selem_close = morphology.ball(selem_close_radius)
# binary_image_closed = morphology.binary_closing(binary_image_opened, footprint=selem_close)

# Choose which morphological output to proceed with
# Start with `binary_image_opened`, as your primary goal is to separate.
processed_binary_image = binary_image_opened # Or binary_image_closed, or just binary_image


In [47]:
# --- Step 3C: Watershed Segmentation ---
# This is crucial for separating truly connected components.

# Ensure `processed_binary_image` is boolean (True/False) for distance transform
processed_binary_image = processed_binary_image.astype(bool)

# 1. Compute Euclidean Distance Transform
#   Pixels inside the foreground objects have values representing their distance to the nearest background pixel.
#   Peaks in this transform correspond to potential object centers.
distance_map = ndi.distance_transform_edt(processed_binary_image)

# 2. Find Markers (Regional Maxima) for Watershed
#   These are the "seeds" from which the watershed algorithm will grow.
#   `min_distance`: Minimum distance between peaks. Adjust carefully to avoid too many or too few markers.
#   `threshold_abs` or `threshold_rel`: Minimum intensity of peaks.
min_peak_distance_3D = 10 # Adjust based on the expected size of your individual spots.
# You might also want a minimum intensity for the peaks
peak_min_intensity_abs = distance_map.max() * 0.30 # Example: 20% of max distance

local_maxi = feature.peak_local_max(
    distance_map,
    footprint=np.ones((min_peak_distance_3D, min_peak_distance_3D, min_peak_distance_3D)), # Neighborhood for peak finding
    labels=processed_binary_image, # Only find peaks within the foreground
    threshold_abs=peak_min_intensity_abs
)
# Create a markers array from the coordinates of the local maxima
markers = np.zeros(image.shape, dtype=bool)
markers[tuple(local_maxi.T)] = True
# Label the markers uniquely for watershed
labeled_markers, num_markers = ndi.label(markers)

print(f"Found {num_markers} initial markers for watershed.")

# 3. Apply Watershed
#   The watershed algorithm segments the image based on the markers.
#   It 'floods' from the markers, and the 'watershed lines' become the boundaries between segments.
#   `mask`: Constrains the flooding to only within the `processed_binary_image` area.
#   `compactness`: Optional, can make the watershed regions more compact, useful for noisy data.
#   `watershed_line`: If True, returns 0 for watershed lines (boundaries).
segmented_labels = segmentation.watershed(
    -distance_map, # Watershed works on 'basins', so we invert the distance map
    labeled_markers,
    mask=processed_binary_image,
    # compactness=0.1, # Experiment with compactness if you get jagged boundaries
    watershed_line=True # Recommended to get clear boundaries
)

# Post-processing: Remove very small segments (noise)
# This is an optional but often useful step
min_segment_volume = 50 # Adjust based on the smallest expected volume of an SDC body
labels_after_filtering = np.copy(segmented_labels)

# Iterate through unique labels (excluding background 0)
for label in np.unique(segmented_labels):
    if label == 0: # Skip background
        continue
    mask = (segmented_labels == label)
    if np.sum(mask) < min_segment_volume:
        labels_after_filtering[mask] = 0 # Assign to background

print(f"Segments before filtering: {len(np.unique(segmented_labels)) - 1}")
print(f"Segments after filtering: {len(np.unique(labels_after_filtering)) - 1}")

final_segmented_image = labels_after_filtering


Found 103 initial markers for watershed.
Segments before filtering: 103
Segments after filtering: 95


# Visualization

In [48]:
# --- Step 4: Interactive Visualization with Napari ---

viewer = napari.Viewer()

# Add the original image
viewer.add_image(image, name='Original Image', colormap='gray', blending='additive')

# Add binary images at different stages for comparison
viewer.add_labels(binary_image_global, name='Binary - Global Otsu', opacity=0.3, visible=False)
# viewer.add_labels(binary_image_adaptive, name='Binary - Adaptive', opacity=0.3) # Uncomment if you used adaptive

viewer.add_labels(binary_image_opened, name=f'Binary - Opened (selem_radius={selem_radius})', opacity=0.4, visible=False)
#viewer.add_labels(binary_image_closed, name=f'Binary - Closed (selem_close_radius={selem_close_radius})', opacity=0.4, visible=False)

# Add the distance map (useful for understanding watershed markers)
viewer.add_image(distance_map, name='Distance Map', colormap='magma', blending='additive', visible=False) # Start hidden

# Add the markers
viewer.add_points(
    local_maxi,
    name='Watershed Markers',
    size=3, # Adjust point size for visibility
    face_color='red',
    #edge_color='white',
    # ndim=3, # Not necessary if local_maxi is (N, 3)
    opacity=0.7,
    symbol='o',
    visible=False # Start hidden, show when needed
)

# Add the final segmented labels
# Napari will auto-assign random colors to labels, which is great for visualization.
viewer.add_labels(final_segmented_image, name='Final Segmented SDC Bodies', opacity=0.6)

# To keep the napari viewer open
if __name__ == '__main__':
    napari.run()


In [41]:
# --- Step 5: Saving Processed Data for Deep Learning Training ---

# Define a path to save your processed data
base_name= os.path.splitext(image_path)[0]
output_labels_path = os.path.join(os.path.dirname(image_path), f'{base_name}_labels.tif')
output_centroids_path = os.path.join(os.path.dirname(image_path), f'{base_name}_centroids.csv')

# Save the segmented labels as a 3D TIFF stack
# Ensure the data type is appropriate for labels (e.g., uint16, uint32)
# If you have many labels (>2^16-1), use uint32
io.imsave(output_labels_path, final_segmented_image.astype(np.uint16))
print(f"Saved final segmented labels to: {output_labels_path}")

# Extract centroids for SpotiFlow

# Get properties for each labeled region (excluding background label 0)
regions = regionprops(final_segmented_image)
centroids_list = []
for prop in regions:
    # Centroid gives (Z, Y, X) coordinates for 3D
    centroids_list.append({
        'axis-0': prop.centroid[0],
        'axis-1': prop.centroid[1],
        'axis-2': prop.centroid[2]
        #'area_volume': prop.area # For 3D, 'area' is actually volume in pixels
    })

if centroids_list:
    centroids_df = pd.DataFrame(centroids_list)
    centroids_df.to_csv(output_centroids_path, index=False)
    print(f"Saved centroids to: {output_centroids_path}")
else:
    print("No SDC bodies detected to save centroids.")

Saved final segmented labels to: 20241107_1268_E_30minHS_3h_5min_5um_n2v_t26_labels.tif
Saved centroids to: 20241107_1268_E_30minHS_3h_5min_5um_n2v_t26_centroids.csv


INFO: Selected 3 points in this slice, use Shift-A to select all points on the layer. (3 selected)


In [2]:
# --- Reusable Functions Cell ---
# --- Imports ---
import os
import numpy as np
import skimage.io as io
import skimage.filters as filters
import skimage.morphology as morphology
import skimage.measure as measure
import skimage.segmentation as segmentation
import skimage.feature as feature
import scipy.ndimage as ndi
import bioio
import bioio_tifffile
import napari
from skimage.measure import regionprops
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

def preprocess_and_segment_image(
    image_path: str,
    selem_radius: int = 4,
    min_peak_distance_3D: int = 10,
    peak_min_intensity_factor: float = 0.30,
    min_segment_volume: int = 50,
    viewer: napari.Viewer = None,
    save_outputs: bool = True,
) -> tuple:
    
    print(f"\nProcessing image: {os.path.basename(image_path)}")

    # Load image
    img_3D = bioio.BioImage(image_path, reader=bioio_tifffile.Reader)
    # Try to keep the original data type or convert to uint16/uint8 if sufficient range
    # ONLY convert to float32 if pixel values are expected to be float or range > uint16 max.
    # For typical microscopy, uint16 is usually fine for raw data.
    image = img_3D.get_image_data("ZYX", C=1)

    # Normalize to 0-1 range for robust thresholding, but keep as original dtype if possible
    # Or, if intensity range is limited (e.g., 0-255), consider converting to uint8
    # If your images are always uint16, let's stick to that for memory.
    # If you need floating point operations later, convert *only* when necessary.
    if image.dtype != np.float32:
        # Check max value. If within uint16 range (0-65535), keep uint16.
        # If your images have negative values or very large values, float is needed.
        if image.max() > np.iinfo(np.uint16).max or image.min() < 0:
             # Fallback to float32 only if absolutely necessary
            print("Warning: Image values outside uint16 range, converting to float32.")
            image = image.astype(np.float32)
        else:
            image = image.astype(np.uint16) # Use uint16 to save memory

    print(f"Loaded image shape: {image.shape}")
    print(f"Image data type: {image.dtype}") # Check if it's now uint16
    print(f"Image min/max intensity: {image.min()}/{image.max()}")

    # Step 3A: Binarization
    threshold_value = filters.threshold_otsu(image)
    binary_image_global = image > threshold_value
    print(f"Otsu threshold: {threshold_value}")
    binary_image = binary_image_global # This will be bool (1 byte/pixel)

    # Step 3B: Morphological operations
    selem = morphology.ball(selem_radius)
    binary_image_opened = morphology.binary_opening(binary_image, footprint=selem)
    processed_binary_image = binary_image_opened # Still bool

    # Step 3C: Watershed Segmentation
    processed_binary_image = processed_binary_image.astype(bool) # Ensure it's bool for ndi.distance_transform_edt
    # ndi.distance_transform_edt typically returns float64. This is a big one.
    distance_map = ndi.distance_transform_edt(processed_binary_image)

    # Option: Convert distance_map to float32 if float64 is too much
    # This will halve its memory usage. Check if accuracy is sufficient.
    distance_map = distance_map.astype(np.float32)

    peak_min_intensity_abs = distance_map.max() * peak_min_intensity_factor

    local_maxi = feature.peak_local_max(
        distance_map,
        footprint=np.ones((min_peak_distance_3D,) * 3),
        labels=processed_binary_image, # Pass original binary image as label mask
        threshold_abs=peak_min_intensity_abs
    )

    markers = np.zeros(image.shape, dtype=bool)
    if local_maxi.size > 0:
        markers[tuple(local_maxi.T)] = True
    labeled_markers, num_markers = ndi.label(markers) # labeled_markers will be int32 or int64

    print(f"Found {num_markers} initial markers for watershed.")

    if num_markers == 0:
        print("No markers found for watershed. Returning empty results.")
        final_segmented_image = np.zeros_like(image, dtype=np.uint16)
        centroids_df = pd.DataFrame(columns=['axis-0', 'axis-1', 'axis-2'])
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        if viewer:
            viewer.add_image(image, name=f'{base_name} - Original Image', colormap='gray', blending='additive')
            viewer.add_labels(final_segmented_image, name=f'{base_name} - Final Segmented SDC Bodies (No Markers)', opacity=0.6)
        return final_segmented_image, centroids_df, base_name

    segmented_labels = segmentation.watershed(
        -distance_map,
        labeled_markers,
        mask=processed_binary_image,
        watershed_line=True
    )
    # Convert segmented_labels to a smaller integer type if possible.
    # Max label value will determine required dtype. If you have < 65535 segments, uint16 is fine.
    # If segments can exceed this, uint32 is next.
    if segmented_labels.max() < np.iinfo(np.uint16).max:
        segmented_labels = segmented_labels.astype(np.uint16)
    elif segmented_labels.max() < np.iinfo(np.uint32).max:
        segmented_labels = segmented_labels.astype(np.uint32)


    labels_after_filtering = np.copy(segmented_labels) # This creates a copy, doubles memory for labels temporarily
    # Consider directly modifying segmented_labels if you don't need the original pre-filtered version
    # Or, filter in-place to avoid copy:
    # region_sizes = measure.regionprops(segmented_labels)
    # for prop in region_sizes:
    #     if prop.area < min_segment_volume:
    #         labels_after_filtering[segmented_labels == prop.label] = 0


    # More memory-efficient way to filter labels
    unique_labels = np.unique(segmented_labels)
    # Create a mapping for labels to keep (or set to 0)
    label_map = np.zeros(segmented_labels.max() + 1, dtype=segmented_labels.dtype)
    valid_labels = []

    for label in unique_labels:
        if label == 0:
            continue # Skip background
        mask = (segmented_labels == label)
        if np.sum(mask) >= min_segment_volume:
            label_map[label] = label # Keep this label
            valid_labels.append(label)
        # else: label_map[label] remains 0

    # Apply the mapping
    final_segmented_image = label_map[segmented_labels]

    print(f"Segments before filtering: {len(unique_labels) - 1}")
    print(f"Segments after filtering: {len(valid_labels)}") # Adjusted count



    #final_segmented_image = labels_after_filtering
    
    # --- New Section: Post-Watershed Merging for 1-pixel separation ---
    # Define a small structuring element for closing.
    # A 3x3x3 ball (radius 1) or a small cube is often suitable for bridging 1-pixel gaps.
    # Adjust this 'merge_selem_radius' based on how much "merging" you want to allow.
    # For exactly 1-pixel gaps, a radius of 1 (3x3x3 ball) is appropriate.
    merge_selem_radius = 1
    merge_selem = morphology.ball(merge_selem_radius)

    # Convert the segmented image back to a binary mask for closing
    # Any non-zero pixel is considered foreground.
    binary_from_labels = final_segmented_image > 0

    # Apply morphological closing to bridge small gaps
    # This will connect regions that are separated by a small distance.
    closed_binary = morphology.binary_closing(binary_from_labels, footprint=merge_selem)

    # Re-label the connected components after closing
    # This will assign new labels to the merged regions.
    merged_labels, num_merged_segments = ndi.label(closed_binary)

    # Important: We need to ensure that the merged labels correspond to the original
    # object boundaries as much as possible, and that small regions aren't accidentally
    # merged into very large ones if they were far apart but the closing connected them.
    # A common way to refine this is to transfer the properties of the *old* labels
    # to the *new* labels, or to ensure that the new labels are within the original
    # processed_binary_image boundaries.

    # A more robust approach might be to use the original distance map or markers
    # with the merged binary image. However, for a simple "merge 1-pixel apart"
    # using closing, we can relabel and then apply the min_segment_volume again.

    # --- Refinement after closing and relabeling ---
    # The `merged_labels` might have new labels that are smaller than the original.
    # We should re-filter by volume.
    unique_merged_labels = np.unique(merged_labels)
    final_final_segmented_image = np.zeros_like(image, dtype=np.uint16)
    new_label_counter = 0

    print(f"Segments after initial filtering and before final merge filter: {num_merged_segments}")

    for label in unique_merged_labels:
        if label == 0:
            continue
        mask = (merged_labels == label)
        if np.sum(mask) >= min_segment_volume: # Re-apply volume filter
            new_label_counter += 1
            final_final_segmented_image[mask] = new_label_counter
        # else: this merged region is too small, discard it.

    print(f"Segments after post-watershed merge and final filtering: {new_label_counter}")

    final_segmented_image = final_final_segmented_image # Use the newly merged and filtered labels



    # Visualization (optional)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    if viewer:
        viewer.add_image(image, name=f'{base_name} - Original image', colormap='gray', blending='additive')
        viewer.add_labels(binary_image_global, name=f'{base_name} - Binary global Otsu', opacity=0.3, visible=False)
        viewer.add_labels(binary_image_opened, name=f'{base_name} - Binary opened (selem_radius={selem_radius})', opacity=0.4, visible=False)
        viewer.add_image(distance_map, name=f'{base_name} - Distance map', colormap='magma', blending='additive', visible=False)
        viewer.add_points(
            local_maxi,
            name=f'{base_name} - Watershed markers',
            size=3,
            face_color='red',
            opacity=0.7,
            symbol='o',
            visible=False
        )
        viewer.add_labels(final_segmented_image, name=f'{base_name} - Final Segmented SDC Bodies', opacity=0.6)

    # Saving Processed Data
    centroids_list = []
    regions = regionprops(final_segmented_image)
    for prop in regions:
        centroids_list.append({
            'axis-0': prop.centroid[0],
            'axis-1': prop.centroid[1],
            'axis-2': prop.centroid[2]
        })

    if centroids_list:
        centroids_df = pd.DataFrame(centroids_list)
    else:
        centroids_df = pd.DataFrame(columns=['axis-0', 'axis-1', 'axis-2'])

    if save_outputs:
        base_output_dir = os.path.dirname(image_path)
        output_dir = os.path.join(base_output_dir, 'authomated_annotations')
        os.makedirs(output_dir, exist_ok=True)
        output_labels_path = os.path.join(output_dir, f'{base_name}_labels.tif')
        output_centroids_path = os.path.join(output_dir, f'{base_name}.csv')

        io.imsave(output_labels_path, final_segmented_image.astype(np.uint16))
        print(f"Saved final segmented labels to: {output_labels_path}")

        if not centroids_df.empty:
            centroids_df.to_csv(output_centroids_path, index=False)
            print(f"Saved centroids to: {output_centroids_path}")
        else:
            print("No SDC bodies detected to save centroids.")

    return final_segmented_image, centroids_df, base_name


In [5]:

# --- Experimenting with single images (using the reusable function) ---

if __name__ == '__main__':
    # Initialize Napari viewer once for all visualizations
    #viewer = napari.Viewer()

    # Define parameters for the single image experiment
    single_image_filename = '/mnt/external.data/MeisterLab/mvolosko/image_project/SDC1/1268_fast_imaging/spots/raw_images_timelapse/1268_fast_imaging_1_SIM^2processing_t14.tif'
    single_image_params = {
        'selem_radius': 4, #was 4
        'min_peak_distance_3D': 30,  #was 10 and oversegmented
        'peak_min_intensity_factor': 0.20, #was 0.30v
        'min_segment_volume': 100 #was 50
    }

    print("\n--- Running single image experiment ---")
    segmented_single_image, centroids_single_image_df, single_image_base_name = preprocess_and_segment_image(
        image_path=single_image_filename,
        #viewer=viewer,
        save_outputs=True,  # Set to True to save outputs for the single image
        **single_image_params
        )

    print(f"Single image processing complete for: {single_image_base_name}")
    print(f"Number of SDC bodies detected: {len(centroids_single_image_df)}")



--- Running single image experiment ---

Processing image: 1268_fast_imaging_1_SIM^2processing_t14.tif
Loaded image shape: (27, 2560, 2560)
Image data type: uint16
Image min/max intensity: 0/63380
Otsu threshold: 4965
Found 205 initial markers for watershed.
Segments before filtering: 205
Segments after filtering: 199
Segments after initial filtering and before final merge filter: 138
Segments after post-watershed merge and final filtering: 138
Saved final segmented labels to: /mnt/external.data/MeisterLab/mvolosko/image_project/SDC1/1268_fast_imaging/spots/raw_images_timelapse/authomated_annotations/1268_fast_imaging_1_SIM^2processing_t14_labels.tif
Saved centroids to: /mnt/external.data/MeisterLab/mvolosko/image_project/SDC1/1268_fast_imaging/spots/raw_images_timelapse/authomated_annotations/1268_fast_imaging_1_SIM^2processing_t14.csv
Single image processing complete for: 1268_fast_imaging_1_SIM^2processing_t14
Number of SDC bodies detected: 138


In [6]:

# --- Analyse whole folder images ---
if __name__ == '__main__':
    # Define the folder containing images
    image_folder = '/mnt/external.data/MeisterLab/mvolosko/image_project/SDC1/1268_fast_imaging/spots/raw_images_timelapse/'
    # List of image file extensions to process
    image_extensions = ('.tif', '.tiff')

    print("\n--- Running batch processing for folder ---")
    processed_results = []

    for filename in os.listdir(image_folder):
        if filename.lower().endswith(image_extensions):
            full_image_path = os.path.join(image_folder, filename)
            # Process each image using the reusable function
            segmented_img, centroids_df, img_base_name = \
                preprocess_and_segment_image(
                    image_path=full_image_path,
                    save_outputs=True, # Set to True to save outputs for each image in the folder
                    selem_radius= 4,
                    min_peak_distance_3D= 20,
                    peak_min_intensity_factor= 0.20,
                    min_segment_volume= 100
                )
            processed_results.append({
                'filename': img_base_name,
                'segmented_image': segmented_img,
                'centroids': centroids_df,
                'num_sdc_bodies': len(centroids_df)
            })

    print("\n--- Batch processing complete ---")
    for result in processed_results:
        print(f"Image: {result['filename']}, Detected SDC Bodies: {result['num_sdc_bodies']}")

    # # Keep napari viewer open
    # napari.run()


--- Running batch processing for folder ---

Processing image: 1268_fast_imaging_3_SIM^2processing_t18.tif
Loaded image shape: (27, 2560, 2560)
Image data type: uint16
Image min/max intensity: 0/42146
Otsu threshold: 914
Found 2202 initial markers for watershed.
Segments before filtering: 2202
Segments after filtering: 1986
Segments after initial filtering and before final merge filter: 1379
Segments after post-watershed merge and final filtering: 1365
Saved final segmented labels to: /mnt/external.data/MeisterLab/mvolosko/image_project/SDC1/1268_fast_imaging/spots/raw_images_timelapse/authomated_annotations/1268_fast_imaging_3_SIM^2processing_t18_labels.tif
Saved centroids to: /mnt/external.data/MeisterLab/mvolosko/image_project/SDC1/1268_fast_imaging/spots/raw_images_timelapse/authomated_annotations/1268_fast_imaging_3_SIM^2processing_t18.csv

Processing image: 1268_fast_imaging_3_SIM^2processing_t16.tif
Loaded image shape: (27, 2560, 2560)
Image data type: uint16
Image min/max inte

: 